## Table of Contents
###  [1.Pre-Processing](pre-processing.ipynb)
###  [2.Data Assimilation](Data Assimilation.ipynb)
###  [3.Post-Processing](post-processing.ipynb)
<div class="toc" style="margin-top: 1em;">
   <ul class="toc-item" id="toc-level0">
      <li><span><a href='#part 3.1' data-toc-modified-id="part 1"><span class="toc-item-num">&nbsp;&nbsp;</span>3.1 Plot hydraulic conductivity </a></span></li>
      <li><span><a href='#part 3.2' data-toc-modified-id="part 2"><span class="toc-item-num">&nbsp;&nbsp;</span>3.2 Calculate the vertical hydraulic exchange flux (HEF)</a></span></li>
      <li><span><a href='#part 3.3' data-toc-modified-id="part 3"><span class="toc-item-num">&nbsp;&nbsp;</span>3.3 Compare the calculated HEF with that from temperature method</a></span></li>
</div>





<a id='part 3.1'></a>
# 3.1 Plot hydraulic conductivity 
1. Hydraulic conductivity near the riverbed is dynamic over time because upwelling and downwelling would change the sedimentary structure.

In [ ]:
load(paste("results/",tail(system("ls -rt results/",intern=TRUE),1),sep=""))

ntime = length(perm.ls)



perm.ls = as.data.frame(perm.ls)
perm.ls = log10(perm.ls*3600*24*9.81*1000*1000)
colors = rainbow(nreaz)

jpeg(paste("figures/perm.jpg",sep=''),width=8,height=5,units='in',res=300,quality=100)
par(mgp=c(1.8,0.6,0))
plot((1:ntime),perm.ls[1,],type="l",
                                        #     ylim=range(perm.ls),
     ylim = c(-5,5),
     xlab="Time (hr)",
     ylab="Hydraulic conductivity (log10 m/d)"
     )
for (ireaz in 1:nreaz)
{
    lines(1:ntime,perm.ls[ireaz,],type="l",col="lightsalmon3")

}
lines(1:ntime,as.numeric(colMeans(perm.ls)),lwd=2)
## lines(1:length(ref.perm),ref.perm,col="blue",lty=2,lwd=2)

legend("topright",
       c("Mean","Realizations"),
       col=c("black","lightsalmon3"),
       lty=c(1,1),
       bty="n"
       )
dev.off()


![Estimated hydraulic conducivity](./Doc/temp/Picture4.jpg)


<a id='part 3.2'></a>
# 3.2 Plot the vertical hydraulic exchange flux (HEF)
1. the vertical HEF is calculated by Darcy's law: HEF = hydraculic conductivity * hydraulic gradient


In [ ]:
rm(list=ls())
load("/global/cscratch1/sd/xhsong/1d_flux/esmda/data/oneD_input.r")
#setwd("/global/cscratch1/sd/xhsong/1d_flux/esmda/3d/overall_iter_4_sd1/123")
load("results/flux.data.r")


oneD.time = oneD.time/3600  # change to mean
oneD.vz = oneD.vz*24
oneD.vy = oneD.vy*24
oneD.vx = oneD.vx*24
mean.oneD.vz = as.numeric(rowMeans(oneD.vz))
mean.flux = as.numeric(colMeans(flux.data))

oneD.cond = mean.oneD.vz/
    ((oneD.head[,ncol(oneD.head)]-oneD.head[,1])/
     abs(oneD.depth[ncol(oneD.head)]-oneD.depth[1]))

jpeg(paste("figures/flux.jpg",sep=''),width=8,height=5,units='in',res=300,quality=100)
par(mgp=c(1.8,0.6,0))
plot(output.times,
     as.numeric(colMeans(flux.data)),
     lwd=2,
     type="l",
     col="red",
     ylim=c(-10,10),
     ylab="Darcy flux (m/d)",
     xlab="Simulation time (hr)"     
     )
for (ireaz in 1:nreaz)
{
    lines(output.times,flux.data[ireaz,],type="l",col="lightsalmon3")    
}

points(oneD.time,mean.oneD.vz,col="blue",cex=0.5)
lines(output.times,mean.flux,lwd=2,col="black")
legend("topleft",
       c("Reference","Mean","Realizations"),
       col=c("blue","black","lightsalmon3"),
       lty=c(NA,1,1),
       pch=c(1,NA,NA),
       bty="n"
       )
dev.off()


one2one.range = c(24*7,720-24)

simu.one2one.index = which(output.times >= one2one.range[1] &
                           output.times <= one2one.range[2] )
oneD.one2one.index = which(oneD.time >= one2one.range[1] &
                           oneD.time <= one2one.range[2] )




jpeg(paste("figures/flux_vs.jpg",sep=''),width=4.4,height=5,units='in',res=300,quality=100)
par(mgp=c(1.8,0.6,0))
smoothScatter(mean.oneD.vz[simu.one2one.index],mean.flux[simu.one2one.index],
     col="black",
     ylim=c(-3,2),
     xlim=c(-3,2),     
     ylab="Assimilated Darcy flux (m/d)",
     xlab="Reference Darcy flux (m/d)",
     cex=0.5,
     nrpoints=20,
     pch=16,
     asp=1,
     )

lines(-100:100,-100:100,lwd=2,col="red")
dev.off()


load(paste(tail(system("ls -rt results/enkf*",intern=TRUE),1),sep=""))
ntime = length(perm.ls)
perm.ls = as.data.frame(perm.ls)
perm.ls = log10(perm.ls*3600*24*9.81*1000*1000)
mean.perm = as.numeric(colMeans(perm.ls))


jpeg(paste("figures/perm.jpg",sep=''),width=8,height=5,units='in',res=300,quality=100)
par(mgp=c(1.8,0.6,0))
plot(1:ntime,mean.perm,type="l",
     ylim = c(-1,3),
     xlab="Time (hr)",
     ylab="Hydraulic conductivity (log10 m/d)"
     )
for (ireaz in 1:nreaz)
{
    lines(1:ntime,perm.ls[ireaz,],type="l",col="lightsalmon3")
}
lines(1:ntime,mean.perm,lwd=2,col="black")
points(oneD.time,log10(oneD.cond),col="blue",cex=0.5)

legend("topleft",
       c("Reference","Mean","Realizations"),
       col=c("blue","black","lightsalmon3"),
       lty=c(NA,1,1),
       pch=c(1,NA,NA),
       bty="n"
       )
dev.off()





jpeg(paste("figures/perm_vs.jpg",sep=''),width=4.4,height=5,units='in',res=300,quality=100)
par(mgp=c(1.8,0.6,0))
smoothScatter(log10(oneD.cond)[simu.one2one.index],mean.perm[simu.one2one.index],
     col="black",
     ylim=c(1,1.8),
     xlim=c(1,1.8),     
     ylab="Assimilated Hydraulic conductivity (log10 m/d)",
     xlab="Reference Hydraulic conductivity (log10 m/d)",
     cex=0.5,
     nrpoints=20,
     pch=16,
     asp=1,
     )

lines(-100:100,-100:100,lwd=2,col="red")
dev.off()


![Estimated flux](./Doc/temp/Picture1.jpg)


<a id='part 3.3'></a>
# 3.3 Compare the calculated HEF with that from temperature method
1. The daily HEF can also be estimated by temperature data only using VFLUX 2 (Ivrine et al., 2015). The VFLUX 2 is writen in Matlab and will be incorporated into the tutorial later. 
